In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os

model = YOLO('best.pt')

test_images_path = '/test/images/'

test_images = [f for f in os.listdir(test_images_path) if f.endswith('.jpg') or f.endswith('.png')]


aruco_measurement = 20  # cm

#The following is the true hip measurements for the 12 3D meshes

measurement_groups = {
    range(95, 158): 71,  
    range(158, 221): 49,  
    range(221, 284): 37, 
    range(284, 347): 55,  
    range(347, 410): 50, 
    range(410, 472): 47,  
    range(28, 90): 60,  
    range(91, 95): 46,
    range(473, 532): 46, 
    range(532, 595): 49,  
    range(595, 658): 43,  
    range(658, 721): 46,  
    range(721, 783): 57   
}

results_list = []

for img_name in test_images:
    img_path = os.path.join(test_images_path, img_name)
    results = model(img_path)

    boxes = results[0].boxes  
    labels = results[0].names  

    confidences = results[0].boxes.conf  

    aruco_len = None
    hip_len = None

    img_number = int(img_name.split('Screenshot')[1].split('_')[0])

    # To find the image with the true measurement corresponding to it.

    true_measurement = None
    for group, measurement in measurement_groups.items():
        if img_number in group:
            true_measurement = measurement
            break

    if true_measurement is None:
        continue 

    for box, label, conf in zip(boxes.xyxy, labels, confidences):
        if conf > 0.5: 
            
            if label == 0:  
                aruco_len = box[2] - box[0]  
            if label == 1:  
                hip_len = box[2] - box[0]  


    # Estimating the hip measurement with the following formula

    if aruco_len is not None and hip_len is not None:
        estimated_hip_measurement = (aruco_measurement * hip_len) / aruco_len
        difference = abs(estimated_hip_measurement - true_measurement)
        
        results_list.append({
            "Image Name": img_name,
            "True Measurement (cm)": true_measurement,
            "Estimated Measurement (cm)": estimated_hip_measurement,

            "Difference (cm)": difference
        })
    else:
        results_list.append({
            "Image Name": img_name,
            "True Measurement (cm)": true_measurement,

            "Estimated Measurement (cm)": "N/A",
            "Difference (cm)": "N/A"
        })

results_df = pd.DataFrame(results_list)

average_difference = results_df["Difference (cm)"].mean()
max_difference = results_df["Difference (cm)"].max()

min_difference = results_df["Difference (cm)"].min()

least_error = results_df.loc[results_df["Difference (cm)"].idxmin()]
most_error = results_df.loc[results_df["Difference (cm)"].idxmax()]

print(f"Average Difference: {average_difference:.2f} cm")
print(f"Maximum Error: {max_difference:.2f} cm")

print(f"Minimum Error: {min_difference:.2f} cm")

plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.plot(results_df.index, results_df["True Measurement (cm)"], label="True Measurement", color='blue', marker='o')

plt.title("True Measurements")
plt.xlabel("Image Index")

plt.ylabel("Measurement (cm)")

plt.subplot(1, 3, 2)

plt.plot(results_df.index, results_df["Estimated Measurement (cm)"], label="Estimated Measurement", color='red', marker='x')
plt.title("Estimated Measurements")

plt.xlabel("Image Index")
plt.ylabel("Measurement (cm)")


plt.subplot(1, 3, 3)
plt.plot(results_df.index, results_df["True Measurement (cm)"], label="True Measurement", color='blue', marker='o')
plt.plot(results_df.index, results_df["Estimated Measurement (cm)"], label="Estimated Measurement", color='red', marker='x')
plt.title("True vs Estimated Measurements")

plt.xlabel("Image Index")
plt.ylabel("Measurement (cm)")

plt.figtext(0.5, 0.01, f"Avg. Diff: {average_difference:.2f} cm   Max Error: {max_difference:.2f} cm   Min Error: {min_difference:.2f} cm",
            wrap=True, horizontalalignment='center', fontsize=10)

plt.tight_layout()
plt.show()

print(f"\nLeast Error:\n{least_error}")
print(f"Most Error:\n{most_error}")


ModuleNotFoundError: No module named 'torch'